In [2]:
import graphlab

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [3]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1'] = feature
    
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        for power in range(2, degree+1):
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # assign poly_sframe[name] to be feature^power
            poly_sframe[name] = feature.apply(lambda x: x ** power)
    return poly_sframe

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

[INFO] This non-commercial license of GraphLab Create is assigned to kaviarasu.govindaraju@snapchat.com and will expire on February 06, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-16255 - Server binary: /Users/kaviarasu.govindaraju/anaconda/envs/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1458798311.log
[INFO] GraphLab Server Version: 1.8.1


In [5]:
sales = graphlab.SFrame('kc_house_data.gl/')
sales = sales.sort(['sqft_living','price'])

In [6]:
l2_small_penalty = 1e-5

In [7]:
# See co-efficients of 15th degree polynomial
poly15_data = polynomial_sframe(sales['sqft_living'], 15)
poly_features = poly15_data.column_names()
poly15_data['price'] = sales['price']

poly15_model = graphlab.linear_regression.create(dataset=poly15_data, target='price', features=poly_features,
                                                 l2_penalty=l2_small_penalty, validation_set=None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 1.030851     | 2662555.737431     | 245656.462164 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:


In [8]:
poly15_model.coefficients

name,index,value,stderr
(intercept),None,167924.865885,nan
power_1,None,103.090927005,nan
power_2,None,0.134604578336,nan
power_3,None,-0.000129071379155,nan
power_4,None,5.18929004052e-08,nan
power_5,None,-7.77169392624e-12,nan
power_6,None,1.71144978155e-16,nan
power_7,None,4.51177772785e-20,nan
power_8,None,-4.78838025654e-25,nan
power_9,None,-2.3334351196e-28,nan


## Observe Overfitting

In [9]:
# Split the data into split the sales data into four subsets of roughly equal size and call them
# set_1, set_2, set_3, and set_4. Use .random_split function and make sure you set seed=0
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

In [10]:
def train_model(dataset, degree, l2_small_penalty):
    poly_data = polynomial_sframe(dataset['sqft_living'], degree)
    poly_features = poly_data.column_names()
    poly_data['price'] = dataset['price']

    poly_model = graphlab.linear_regression.create(dataset=poly_data, target='price', features=poly_features,
                                                 l2_penalty=l2_small_penalty, validation_set=None)
    return poly_model

In [11]:
set1_model = train_model(set_1, 15, 1e-9)
set2_model = train_model(set_2, 15, 1e-9)
set3_model = train_model(set_3, 15, 1e-9)
set4_model = train_model(set_4, 15, 1e-9)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5404
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.038817     | 2357654.045565     | 247981.976053 |
PROGRESS: | 2         | 3        | 0.076685     | 2357658.307592     | 247981.876675 |
PROGRESS: | 3         | 4        | 0.115344     | 2357658.332788     | 247981.876343 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Opti

In [12]:
def get_coefficient(model, coefficient):
    coeffs = model.coefficients
    filteres_coeff = coeffs[coeffs['name'] == coefficient]
    return filteres_coeff

In [13]:
coefficient = 'power_1'
get_coefficient(set1_model, coefficient)

name,index,value,stderr
power_1,None,798.699985535,2980.70926946


In [14]:
get_coefficient(set2_model, coefficient)

name,index,value,stderr
power_1,None,-4232.7143182,7872.16188135


In [15]:
get_coefficient(set3_model, coefficient)

name,index,value,stderr
power_1,None,2946.38118365,6149.34446173


In [16]:
get_coefficient(set4_model, coefficient)

name,index,value,stderr
power_1,None,-775.713132074,37810.2426296


## Ridge Regression Comes To Rescue

In [17]:
set1_ridge_model = train_model(set_1, 15, 1e5)
set2_ridge_model = train_model(set_2, 15, 1e5)
set3_ridge_model = train_model(set_3, 15, 1e5)
set4_ridge_model = train_model(set_4, 15, 1e5)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5404
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.045155     | 5978778.434729     | 374261.720860 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:
PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5398
PRO

In [18]:
get_coefficient(set1_ridge_model, coefficient)

name,index,value,stderr
power_1,None,2.58738875673,4498.57625495


In [19]:
get_coefficient(set2_ridge_model, coefficient)

name,index,value,stderr
power_1,None,2.04470474182,10860.2868263


In [20]:
get_coefficient(set3_ridge_model, coefficient)

name,index,value,stderr
power_1,None,2.26890421877,8587.5632283


In [21]:
get_coefficient(set4_ridge_model, coefficient)

name,index,value,stderr
power_1,None,1.91040938244,50030.7809644


## Selecting an L2 penalty via cross-validation

In [22]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)

In [61]:
# Method to split given dataset for k-fold-validation
def split_k_fold(k, dataset):
    n = len(dataset)
    k_fold_data = []
    for i in xrange(k):
        start = (n*i)/k
        end = (n*(i+1))/k-1
        k_fold_data.append(dataset[start:end+1])
    return k_fold_data

def get_training_data_for_k_fold_model(fold_index, k, dataset):
    n = len(dataset)
    part1_end = (n*fold_index)/k
    part1 = dataset[0:part1_end]
    part2_start = (n*(fold_index+1))/k-1
    part2 = dataset[part2_start+1:n]
    return part1.append(part2)

def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predicted_price = model.predict(data)
    # Then compute the residuals/errors
    residuals = predicted_price - outcome
    # print residuals
    # Then square and add them up
    RSS = (residuals * residuals).sum()
    return(RSS)

In [62]:
validation_groups = split_k_fold(10, train_valid_shuffled)
validation4 = validation_groups[3]

training_data4 = get_training_data_for_k_fold_model(3, 10, train_valid_shuffled)

In [75]:
import numpy as np

# Method to run k fold cross validation and returns the average error
def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    groups = split_k_fold(k, data)
    
    total_error = []
    for i in xrange(k):
        validation_set_i = groups[i]
        training_set = get_training_data_for_k_fold_model(i, k, data)
        model_i = graphlab.linear_regression.create(dataset=training_set, target='price', features=features_list,
                                                 l2_penalty=l2_penalty, validation_set=None, verbose=False)
        total_error.append(get_residual_sum_of_squares(model_i, validation_set_i, validation_set_i[output_name]))
    
    return np.mean(total_error)

In [76]:
data_sframe = polynomial_sframe(train_valid_shuffled['sqft_living'], 15)
feature_list = data_sframe.column_names()
data_sframe['price'] = train_valid_shuffled['price']
penalty_array = np.logspace(1, 7, num=13)

for penalty in penalty_array:
    print "Error for penalty " + str(penalty) + " = " + str(k_fold_cross_validation(10, penalty, data_sframe, 'price', feature_list))

Error for penalty 10.0 = 4.91826427769e+14
Error for penalty 31.6227766017 = 2.87504229919e+14
Error for penalty 100.0 = 1.60908965822e+14
Error for penalty 316.227766017 = 1.22090967326e+14
Error for penalty 1000.0 = 1.21192264451e+14
Error for penalty 3162.27766017 = 1.2395000929e+14
Error for penalty 10000.0 = 1.36837175248e+14
Error for penalty 31622.7766017 = 1.71728094842e+14
Error for penalty 100000.0 = 2.2936143126e+14
Error for penalty 316227.766017 = 2.52940568729e+14
Error for penalty 1000000.0 = 2.58682548441e+14
Error for penalty 3162277.66017 = 2.62819399742e+14
Error for penalty 10000000.0 = 2.64889015378e+14


In [77]:
final_model = train_model(train_valid, 15, 1000.0)
rss = get_residual_sum_of_squares(final_model, test, test['price'])

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 19396
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.025615     | 2461778.986191     | 248914.007014 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:


In [78]:
print rss

2.52897427447e+14
